In [ ]:
import pandas as pd
df =  pd.read_csv(r"C:\Users\Carlo\Desktop\simulador_empresarial\ProjecteData\Equip_16\Data\banca_06.10_dataset.csv")

In [ ]:
#verificación nulos
df.isnull().any().any()

In [ ]:
# suma de nulos por columna
df.isnull().sum()

In [ ]:
# Para ver duplicados

df.duplicated()
# df = df.drop_duplicates(subset=['Product', 'Location']) (para eliminar duplicados concretos en función de una columna)

In [ ]:
df = df.drop_duplicates()

In [ ]:
filas_con_nulos = df[df.isnull().any(axis=1)]
filas_con_nulos

In [ ]:
def clase_edad(x):
    if x < 18:
        return "unknown"
    elif x >= 18 and x <=24: 
        return '18-24'
    elif  x >= 25 and x <= 34:
        return '25-34'
    elif  x >= 35 and x <= 44:
        return '35-44'
    elif  x >= 45 and x <= 54:
        return '45-54'
    elif  x >= 55 and x <= 64:
        return '55-64'
    else:
        return '65+' #Como no hay edades inferiores a 18 no va dar problemae

df['Clase de edad'] = df['age'].apply(clase_edad)

In [ ]:
def perfil(x):
    if x == "unknown":
        return "unknown"
    elif x == '18-24': 
        return "Gen Z & Boomers"
    elif  x == '25-34':
        return 'Millenials'
    elif  x == '35-44':
        return 'Adultxs'
    elif  x == '45-54':
        return 'Divorciadxs'
    elif  x == '55-64':
        return "Gen Z & Boomers"
    elif x == '65+':
        return "Gen Z & Boomers" #Como no hay edades inferiores a 18 no va dar problemae

df["Perfil"] = df['Clase de edad'].apply(perfil)
df # Para visualizar la tabla generada

In [ ]:
df["date"] = pd.to_datetime(
    dict(year=df["year"], month=df["month #"], day=df["day"]),
    errors="coerce"
)

In [ ]:
errores = df[df["date"].isna()]
print(errores)

In [13]:
import calendar

def validar_fecha(y, m, d):
    try:
        return pd.Timestamp(y, m, d)
    except ValueError:
        return pd.NaT

df["date"] = [
    validar_fecha(y, m, d) for y, m, d in zip(df["year"], df["month #"], df["day"])
]

In [14]:
df = df.dropna(subset=["date"])

In [18]:
import calendar

# Asegurarte de que df es una copia independiente
df = df.copy()

def es_fecha_valida(y, m, d):
    try:
        return 1 <= d <= calendar.monthrange(y, m)[1]
    except Exception:
        return False

df.loc[:, "es_valida"] = [
    es_fecha_valida(y, m, d) for y, m, d in zip(df["year"], df["month #"], df["day"])
]

In [20]:
df["date"] = pd.to_datetime(
    dict(year=df["year"], month=df["month #"], day=df["day"]),
    errors="coerce"   # convierte inválidas en NaT
)

In [21]:
df["weekday_num"] = df["date"].dt.weekday
df = df[df["weekday_num"] <= 4]
df["day_week"] = df["date"].dt.day_name(locale="es_ES")

In [22]:
import pandas as pd

meses_numerico = {
    "jan": 1, "feb": 2, "mar": 3, "apr": 4,
    "may": 5, "jun": 6, "jul": 7, "aug": 8,
    "sep": 9, "oct": 10, "nov": 11, "dec": 12
}

df["month #"] = df["month"].map(meses_numerico)

# Detectar "saltos" de año: cuando el mes baja (ej. de dic -> ene)
df["year_offset"] = (df["month #"].diff() < 0).cumsum()

# Año base 2008 + offset
df["year"] = 2008 + df["year_offset"]

# Crear la fecha completa
df["date"] = pd.to_datetime(dict(year=df["year"], month=df["month #"], day=df["day"]))

# Día de la semana (solo lunes a viernes)
df["weekday_num"] = df["date"].dt.weekday
df = df[df["weekday_num"] <= 4]
df["day_week"] = df["date"].dt.day_name(locale="es_ES")

In [23]:
# Creamos un orden de meses empezando en mayo
orden_meses = [5,6,7,8,9,10,11,12,1,2,3,4]

# Convertimos la columna "month #" en categórica con ese orden
df["month #"] = pd.Categorical(df["month #"], categories=orden_meses, ordered=True)

# Reordenamos el DataFrame
df = df.sort_values(["month #", "day"]).reset_index(drop=True)

In [24]:
# -------------------------
# Función para imputar nulos
# -------------------------
def imputar(row):
    # 1) age
    if pd.isna(row["age"]):
        if row["job"] == "retired":
            row["age"] = 65
        elif row["job"] == "management":
            row["age"] = 45
        elif row["job"] == "technician":
            row["age"] = 35
        else:
            row["age"] = 40
    
    # 2) marital
    if pd.isna(row["marital"]):
        row["marital"] = "single" if row["age"] < 32 else "married"
    
    # 3) education
    if pd.isna(row["education"]):
        if row["job"] in ["management", "technician"]:
            row["education"] = "tertiary"
        elif row["job"] == "retired":
            row["education"] = "secondary"
        else:
            row["education"] = "primary"
    
    # 4) housing
    if pd.isna(row["housing"]):
        if row["Clase de edad"] == "25-34":
            row["housing"] = "yes"
        else:
            row["housing"] = "no"
    return row
# Aplicamos la función
df = df.apply(imputar, axis=1)
df.isnull().sum()

id               0
age              0
job              0
marital          0
education        0
default          5
balance          0
housing          0
loan             0
contact          0
day              0
month            0
duration         0
campaign         0
pdays            0
previous         0
poutcome         0
deposit          0
Clase de edad    0
Perfil           0
month #          0
date             0
day_week         0
year_offset      0
year             0
es_valida        0
weekday_num      0
dtype: int64

In [25]:
def change_duplic_id(df):
    # Encuentra las filas duplicadas (solo las segundas y siguientes)
    duplicados = df[df.duplicated(subset=['id'], keep='first')].index

    # Obtén el ID máximo actual
    max_id = df['id'].max()

    # Genera nuevos IDs consecutivos a partir del máximo
    nuevos_ids = range(max_id + 1, max_id + 1 + len(duplicados))

    # Asigna los nuevos IDs a las filas duplicadas
    df.loc[duplicados, 'id'] = nuevos_ids

change_duplic_id(df)

# df.duplicated(subset=['id'])

df[df['id'].duplicated(keep=False)].sort_values('id')

# print(variable==True)






,id,age,job,marital,education,default,balance,housing,loan,contact,...,deposit,Clase de edad,Perfil,month #,date,day_week,year_offset,year,es_valida,weekday_num


In [ ]:
df

In [26]:
df.to_csv(r"C:/Users/Carlo/Desktop/simulador_carlos/datos.csv", index=False, encoding="utf-8-sig")